In [1]:
# MeCabとneologdのインストール
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null
!ln -s /etc/mecabrc /usr/local/etc/mecabrc
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

# wordcloudで使う日本語フォントのインストール
!apt-get -y install fonts-ipafont-gothic

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-ipafont-mincho
The following NEW packages will be installed:
  fonts-ipafont-gothic fonts-ipafont-mincho
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 8,251 kB of archives.
After this operation, 28.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-gothic all 00303-18ubuntu1 [3,526 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-ipafont-mincho all 00303-18ubuntu1 [4,725 kB]
Fetched 8,251

In [16]:
import re
import MeCab
from wordcloud import WordCloud
import matplotlib.pyplot as plt 

tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")

In [28]:
txt_file = 'sample.txt'

In [ ]:
def sanitize_text(text):
    text = re.sub('［.+?］', '', text)
    text = re.sub('《.+?》', '', text)
    text = re.sub('｜', '', text)
    text = re.sub('\s', '', text)
    return text

def get_nouns(text):
    node = tagger.parseToNode(text)
    nouns = []
    while node:
        if 36 <= node.posid <=67: # 名詞のみ取得
            nouns.append(node.surface)
        node = node.next
    return nouns
  
with open(txt_file, encoding='utf-8') as f:
    text = f.read()
text = sanitize_text(text)
nouns = get_nouns(text)
text=" ".join(nouns)

# WordCloudで表示
wordcloud = WordCloud(font_path='/usr/share/fonts/truetype/fonts-japanese-gothic.ttf',
                      width=900,
                      height=600,
                      background_color='white',
                      max_words=300,
                      min_font_size=10,
                      collocations = True).generate(text)
plt.figure(figsize=(15,12))
plt.axis('off')
plt.imshow(wordcloud)
plt.savefig('word_cloud.png')
plt.show()